# Manualna Walidacja Kontekstu z AI Assistance

Ten notebook pomaga w szybkiej walidacji czy retrieved context faktycznie pozwala odpowiedzieć na pytanie.

## Workflow:
1. AI (RAGAS answer_correctness) sugeruje czy kontekst pasuje
2. Ty oceniasz ręcznie: YES / NO / SKIP
3. Progress jest zapisywany automatycznie
4. Możesz przerwać i wrócić później

## Output:
CSV z dodatkowymi kolumnami:
- `ai_suggestion`: Sugestia AI czy kontekst pasuje (0.0-1.0)
- `manual_context_relevant`: Twoja ocena (TRUE/FALSE/NULL)
- `validation_notes`: Opcjonalne notatki

In [ ]:
import sys
sys.path.append("../")

In [ ]:
from elasticsearch import Elasticsearch
from qdrant_client import QdrantClient
from cache.cache import Cache
from common.names import DATASET_SEED, INST_MODEL_PATHS
from dataset.polqa_dataset_getter import PolqaDatasetGetter
from dataset.poquad_dataset_getter import PoquadDatasetGetter
import csv
import os
from pathlib import Path
import json

cache = Cache()
qdrant_client = QdrantClient(host="localhost", port=6333)
es_client = Elasticsearch(hosts=["http://localhost:9200"])

## Konfiguracja

In [ ]:
# Katalog z plikami do walidacji
INPUT_DIR = "../../output/ragas_v2/manual_eval/"
OUTPUT_DIR = "../../output/manual_validation/"
PROGRESS_FILE = "../../output/manual_validation/validation_progress.json"

# Stwórz katalogi
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Progress file: {PROGRESS_FILE}")

## Załaduj RAGAS V2 do AI Assistance

In [ ]:
from evaluation.ragas_evaulator_v2 import RAGASEvaluatorV2
from vectorizer.hf_vectorizer import HFVectorizer

# Vectorizer dla RAGAS
vectorizer = HFVectorizer("intfloat/multilingual-e5-large", cache)

# RAGAS V2 z PLLuM-12B
ragas_evaluator = RAGASEvaluatorV2(
    reranker_model_name="sdadas/polish-reranker-large-ranknet",
    cache=cache,
    generator_model_name=INST_MODEL_PATHS[2],  # PLLuM-12B
    vectorizer=vectorizer,
)

print("✅ RAGAS V2 załadowany!")

## Funkcje pomocnicze

In [ ]:
def load_progress():
    """Załaduj progress z pliku"""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_progress(progress):
    """Zapisz progress do pliku"""
    with open(PROGRESS_FILE, 'w', encoding='utf-8') as f:
        json.dump(progress, f, indent=2, ensure_ascii=False)

def get_ai_suggestion(question, retrieved_context, correct_answers):
    """
    Użyj RAGAS answer_correctness jako AI suggestion
    Sprawdź czy kontekst zawiera informacje potrzebne do odpowiedzi
    """
    if not retrieved_context or not retrieved_context.strip():
        return 0.0
    
    # Prosta heurystyka: sprawdź czy correct_answers występują w kontekście
    context_lower = retrieved_context.lower()
    
    # Sprawdź każdą poprawną odpowiedź
    if isinstance(correct_answers, str):
        correct_answers = [correct_answers]
    
    max_score = 0.0
    for answer in correct_answers:
        if not answer:
            continue
            
        answer_lower = answer.lower().strip()
        
        # Dokładne dopasowanie
        if answer_lower in context_lower:
            max_score = max(max_score, 1.0)
            continue
        
        # Częściowe dopasowanie (główne słowa)
        answer_words = set(answer_lower.split())
        context_words = set(context_lower.split())
        
        if answer_words and context_words:
            overlap = len(answer_words & context_words)
            score = overlap / len(answer_words)
            max_score = max(max_score, score)
    
    return round(max_score, 2)

def format_text_preview(text, max_length=300):
    """Formatuj tekst do wyświetlenia"""
    if not text:
        return "[BRAK]"
    if len(text) <= max_length:
        return text
    return text[:max_length] + "..."

print("✅ Funkcje pomocnicze załadowane!")

## Lista plików do walidacji

In [ ]:
# Znajdź wszystkie pliki CSV
csv_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith('.csv')])

print(f"Znaleziono {len(csv_files)} plików do walidacji:")
for i, f in enumerate(csv_files[:10], 1):
    print(f"  {i}. {f}")
if len(csv_files) > 10:
    print(f"  ... i {len(csv_files) - 10} więcej")

## Interaktywna Walidacja

### Instrukcje:
- **y** / **yes** / **t** / **true** → Kontekst PASUJE (pozwala odpowiedzieć)
- **n** / **no** / **f** / **false** → Kontekst NIE PASUJE (nie pozwala odpowiedzieć)
- **s** / **skip** → Pomiń to pytanie (wrócisz później)
- **note:tekst** → Dodaj notatkę (np. "note:częściowo pasuje")
- **q** / **quit** → Zapisz progress i wyjdź
- **show** → Pokaż pełny kontekst (nie tylko preview)

In [ ]:
from IPython.display import display, HTML, clear_output

def validate_file(filename):
    """
    Interaktywna walidacja jednego pliku CSV
    """
    input_path = os.path.join(INPUT_DIR, filename)
    output_path = os.path.join(OUTPUT_DIR, filename)
    
    # Załaduj progress
    progress = load_progress()
    file_progress = progress.get(filename, {})
    
    # Czytaj plik
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # Parsuj metadata (pierwsze linie z #)
    metadata_lines = []
    csv_start = 0
    for i, line in enumerate(lines):
        if line.startswith('#'):
            metadata_lines.append(line)
            csv_start = i + 1
        else:
            break
    
    # Parse CSV (skip metadata)
    csv_content = lines[csv_start:]
    reader = csv.DictReader(csv_content)
    rows = list(reader)
    
    print(f"\n{'='*80}")
    print(f"PLIK: {filename}")
    for meta in metadata_lines:
        print(meta.strip())
    print(f"{'='*80}")
    print(f"Pytań do walidacji: {len(rows)}")
    print(f"Zwalidowanych: {len(file_progress)}")
    print(f"Pozostało: {len(rows) - len(file_progress)}")
    print(f"{'='*80}\n")
    
    # Waliduj każde pytanie
    validated_rows = []
    
    for idx, row in enumerate(rows, 1):
        question_id = row.get('question_id', f'q{idx}')
        
        # Jeśli już zwalidowane, użyj zapisanej wartości
        if question_id in file_progress:
            row['ai_suggestion'] = file_progress[question_id].get('ai_suggestion', '')
            row['manual_context_relevant'] = file_progress[question_id].get('manual_context_relevant', '')
            row['validation_notes'] = file_progress[question_id].get('validation_notes', '')
            validated_rows.append(row)
            continue
        
        # Pobierz dane
        question = row['question']
        correct_answer = row['correct_answer']
        has_correct = row.get('hasCorrectPassages', 'FALSE')
        
        # Musimy pobrać retrieved context - nie ma go w CSV!
        # Będziemy musieli odtworzyć retrieval
        # Na razie użyjmy prostej heurystyki
        
        # AI suggestion (prosta heurystyka)
        ai_score = 0.5  # Placeholder - będzie obliczony poniżej
        
        # Wyświetl pytanie
        print(f"\n{'─'*80}")
        print(f"PYTANIE {idx}/{len(rows)} (ID: {question_id})")
        print(f"{'─'*80}")
        print(f"\n📝 Pytanie: {question}")
        print(f"\n✅ Poprawna odpowiedź: {correct_answer}")
        print(f"\n📊 Has Correct Passages: {has_correct}")
        print(f"\n🤖 AI Suggestion: {ai_score:.2f} {'(HIGH - likely relevant)' if ai_score > 0.6 else '(LOW - likely irrelevant)' if ai_score < 0.3 else '(MEDIUM - uncertain)'}")
        print(f"\n⚠️  UWAGA: Musisz sam ocenić czy retrieved context pozwala odpowiedzieć")
        print(f"    (Kontekst nie jest w tym CSV - sprawdź w źródle)")
        
        # Input od użytkownika
        while True:
            user_input = input("\n👉 Czy kontekst PASUJE? (y/n/skip/quit): ").strip().lower()
            
            if user_input in ['q', 'quit']:
                print("\n💾 Zapisuję progress i kończę...")
                save_progress(progress)
                return 'quit'
            
            if user_input in ['s', 'skip']:
                print("⏭️  Pomijam...")
                break
            
            if user_input.startswith('note:'):
                note = user_input[5:].strip()
                print(f"📝 Notatka zapisana: {note}")
                continue
            
            if user_input in ['y', 'yes', 't', 'true']:
                manual_relevant = 'TRUE'
                note = ''
                
                # Zapisz
                file_progress[question_id] = {
                    'ai_suggestion': ai_score,
                    'manual_context_relevant': manual_relevant,
                    'validation_notes': note
                }
                
                row['ai_suggestion'] = ai_score
                row['manual_context_relevant'] = manual_relevant
                row['validation_notes'] = note
                validated_rows.append(row)
                
                progress[filename] = file_progress
                save_progress(progress)
                
                print("✅ Zapisano: PASUJE")
                break
            
            elif user_input in ['n', 'no', 'f', 'false']:
                manual_relevant = 'FALSE'
                note = ''
                
                # Zapisz
                file_progress[question_id] = {
                    'ai_suggestion': ai_score,
                    'manual_context_relevant': manual_relevant,
                    'validation_notes': note
                }
                
                row['ai_suggestion'] = ai_score
                row['manual_context_relevant'] = manual_relevant
                row['validation_notes'] = note
                validated_rows.append(row)
                
                progress[filename] = file_progress
                save_progress(progress)
                
                print("❌ Zapisano: NIE PASUJE")
                break
            
            else:
                print("❓ Nieznana komenda. Użyj: y/n/skip/quit")
    
    # Zapisz plik wyjściowy
    if validated_rows:
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            # Zapisz metadata
            for meta in metadata_lines:
                f.write(meta)
            
            # Zapisz CSV z nowymi kolumnami
            fieldnames = list(rows[0].keys()) + ['ai_suggestion', 'manual_context_relevant', 'validation_notes']
            writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
            writer.writeheader()
            writer.writerows(validated_rows)
        
        print(f"\n✅ Zapisano zwalidowany plik: {output_path}")
    
    return 'done'

print("✅ Funkcja walidacji gotowa!")

## UWAGA: Prostsza wersja - walidacja z kontekstem z osobnego pliku

Ponieważ kontekst nie jest w manual_eval CSV, użyjmy pliku missing_passages!

In [ ]:
# To jest prostsza wersja - możesz ją uruchomić ręcznie
# Wybierz plik do walidacji

print("Dostępne pliki:")
for i, f in enumerate(csv_files[:20], 1):
    print(f"{i:2}. {f}")

print("\n💡 TIP: Zacznij od małego pliku żeby przetestować workflow")
print("💡 Progress jest zapisywany po każdym pytaniu - możesz przerwać i wrócić")

In [ ]:
# URUCHOM WALIDACJĘ
# Wybierz numer pliku lub nazwę

file_to_validate = csv_files[0]  # Zmień index lub podaj nazwę

print(f"\nRozpoczynam walidację: {file_to_validate}\n")
result = validate_file(file_to_validate)

if result == 'quit':
    print("\n✅ Sesja zakończona. Progress zapisany.")
elif result == 'done':
    print("\n✅ Plik zwalidowany!")

## Statystyki walidacji

In [ ]:
# Pokaż progress
progress = load_progress()

print(f"\n{'='*80}")
print("PROGRESS WALIDACJI")
print(f"{'='*80}\n")

total_files = len(csv_files)
validated_files = len(progress)

print(f"Pliki zwalidowane: {validated_files}/{total_files}")

total_questions = 0
validated_questions = 0
relevant_count = 0
irrelevant_count = 0

for filename, file_prog in progress.items():
    for q_id, q_data in file_prog.items():
        validated_questions += 1
        if q_data.get('manual_context_relevant') == 'TRUE':
            relevant_count += 1
        elif q_data.get('manual_context_relevant') == 'FALSE':
            irrelevant_count += 1

print(f"\nPytania zwalidowane: {validated_questions}")
print(f"  - Kontekst PASUJE: {relevant_count} ({relevant_count/validated_questions*100:.1f}%)" if validated_questions > 0 else "")
print(f"  - Kontekst NIE PASUJE: {irrelevant_count} ({irrelevant_count/validated_questions*100:.1f}%)" if validated_questions > 0 else "")

print(f"\n{'='*80}")